<a href="https://colab.research.google.com/github/MBayezid/data-conversion-demo-python/blob/main/data_convertion%26hiding_dottedVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 0.0 Imports
import zipfile
import os
import base64
import shutil
from PIL import Image, ImageDraw
import io
from google.colab import files


In [3]:
# @title 0.1 Prompts the user to select or create a directory to store uploaded files. Returns a list of selected file paths.


def select_files():
    """
    Prompts the user to select or create a directory to store uploaded files.
    Returns a list of selected file paths.
    """

    # Get user input for directory path
    while True:
        uploaded_dir = input("Enter directory to store files (or leave empty): ")

        # Use default if input is empty
        if not uploaded_dir:
            uploaded_dir = "uploaded_files"
            break


        # Check if directory exists, create if not
        if not os.path.exists(uploaded_dir):
            create_dir = input(f"Directory '{uploaded_dir}' does not exist. Create it? (y/n): ")
            if create_dir.lower() == 'y' or create_dir.lower()=='':

                break
            else:
                print("Please enter a valid directory or choose to create one.")
        else:
            break

    # Change to the selected directory
    os.makedirs(uploaded_dir, exist_ok=True)
    # os.chdir(uploaded_dir)
    print(f"Uploading to: {uploaded_dir}")

    # Upload files
    print("Select files to upload:")
    uploaded = files.upload(target_dir = uploaded_dir)
    return list(uploaded.keys())

In [5]:
# Example usage with file selection:
file_paths = select_files()

Enter directory to store files (or leave empty): 
Uploading to: uploaded_files
Select files to upload:


Saving lemonade_images.zip to uploaded_files/lemonade_images.zip
Saving logcat.txt to uploaded_files/logcat.txt
Saving Oniket prantor অনিকেত প্রান্তর  Artcell  Tobu ei deyaler shorire  duto manchitro by artcell.m4a to uploaded_files/Oniket prantor অনিকেত প্রান্তর  Artcell  Tobu ei deyaler shorire  duto manchitro by artcell.m4a
Saving Simple.odp to uploaded_files/Simple.odp
Saving trend_icon.png to uploaded_files/trend_icon.png


In [6]:
 # @title 1.0 Zips multiple files into a single zip archive.

def zip_files(file_paths, zip_filename, ziped_out_dir):
  """
  Zips multiple files into a single zip archive.

  Args:
    file_paths: A list of file paths to be zipped.
    zip_filename: The name of the output zip file (including .zip extension).
    ziped_out_dir: The directory where the zip file will be saved.
  """
  try:
    if not zip_filename.endswith(".zip"):
      zip_filename = zip_filename + ".zip"

    full_zip_path = os.path.join(ziped_out_dir, zip_filename)
    os.makedirs(ziped_out_dir, exist_ok=True)

    with zipfile.ZipFile(full_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
      for file_path in file_paths:
        if os.path.exists(file_path):
          zipf.write(file_path, arcname=os.path.basename(file_path))
        else:
          print(f"Warning: File not found: {file_path}")
    print(f"Successfully created {full_zip_path}")
    return full_zip_path

  except FileNotFoundError:
    print(f"Error: One or more files not found.")
  except Exception as e:
    print(f"An error occurred: {e}")



# Example Usage:
if file_paths:
  zip_filename = input("Enter zip name: ")

  if not zip_filename or zip_filename=="":
    zip_filename = "my_archive.zip"

  zipped_out_dir =  "zipped_out_dir"
  full_zip_path = zip_files(file_paths, zip_filename, zipped_out_dir)

else:
  print("No files selected.")

Enter zip name: 
Successfully created zipped_out_dir/my_archive.zip


In [7]:
# @title 2.0 Converts a zip file to a base64 encoded string.

def zip_to_base64(zip_filename):
    """Converts a zip file to a base64 encoded string."""
    try:
        with open(zip_filename, "rb") as zip_file:
            zip_data = zip_file.read()
        encoded_zip = base64.b64encode(zip_data).decode('utf-8')
        return encoded_zip
    except FileNotFoundError:
        print(f"Error: Zip file not found: {zip_filename}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [8]:
# Example usage with file selection:
zip_to_base64_text = zip_to_base64(full_zip_path)
print(len(zip_to_base64_text))
zip_to_base64_text[:50]

5009180


'UEsDBBQAAAAIAHmDYVoMojKBOCcAAKMpAAATAAAAbGVtb25hZG'

In [9]:
# @title  2.1 Converts a base64 into its binary representation.

def base64_to_binary(base64_string):
    """Converts a base64 encoded string to its binary representation."""
    try:
        decoded_bytes = base64.b64decode(base64_string)
        binary_string = ''.join(format(byte, '08b') for byte in decoded_bytes)
        return binary_string
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [10]:
# Example usage with file selection:
base64_to_binary_text = base64_to_binary(zip_to_base64_text)
print(len(base64_to_binary_text))
base64_to_binary_text[:50]

30055072


'01010000010010110000001100000100000101000000000000'

In [11]:
# @title 3.0 Pre-Calculations

# Frame Shape FS = width*height*channels
width, height, channels = 255, 255, 1
FS = width*height*channels
print("\nFrame Shape (FS) = width(255)*height(255)*channels(1) : ", FS)

# Single Frame Size in Pixels (SFSP) = ((steps*width)*(steps*height))* channels
# steps/grid is the size of each squre shaped pixels in our generated frames
steps=3
print("\nSteps/Grid size sat to: ",steps)

width*=steps
print("\tnew width = width*steps: ",width)
height*=steps
print("\tnew height = height*steps: ",height)

FS = width*height*channels
print("\nNew Frame Shape (FS) will be: ((steps*width)*(steps*height))* channels : ", FS)

# SFSP=((steps*width)*(steps*height))*channels
# print("\nSingle Frame Size in Pixels (SFSP) = ((steps*width)*(steps*height))* channels : ", SFSP)

SFC = FS/(steps*steps)
print("\nSingle Frame Capacity (SFC) = FS / (steps*steps) : ", SFC)

# ---------------------------------------------
#
print("\nLength of our string when, base64  : ", len(zip_to_base64_text))
print("Length of our string when, binary : ", len(base64_to_binary_text))
print("\tLength diffrence between base64 and binary: ",(len(zip_to_base64_text)-len(base64_to_binary_text)))


DS = len(base64_to_binary_text)
print("\nData Size / length of binary_string: (DS): " ,DS )

# Frames Number or to be generate (FN)=  DS/SFC
FN =  DS/SFC
print("\nFrames Number or to be generate (FN) =  DS/SFC : " ,FN)
if FN%2 != 0:
  FN = int(FN)+1
else:
  FN = int(FN)
print("\nFN cant be fruction/float >>>> FN = int(FN)+1 ")
print("\nFrames Number or to be generate (FN) =  DS/SFC : " ,FN)





Frame Shape (FS) = width(255)*height(255)*channels(1) :  65025

Steps/Grid size sat to:  3
	new width = width*steps:  765
	new height = height*steps:  765

New Frame Shape (FS) will be: ((steps*width)*(steps*height))* channels :  585225

Single Frame Capacity (SFC) = FS / (steps*steps) :  65025.0

Length of our string when, base64  :  5009180
Length of our string when, binary :  30055072
	Length diffrence between base64 and binary:  -25045892

Data Size / length of binary_string: (DS):  30055072

Frames Number or to be generate (FN) =  DS/SFC :  462.2079507881584

FN cant be fruction/float >>>> FN = int(FN)+1 

Frames Number or to be generate (FN) =  DS/SFC :  463


In [12]:
import numpy as np

def create_grayscale_images_from_binary_string(binary_string, width, height, max_pixels_per_image, output_folder="output_images"):
  """
 # @title 4.0 Generate multiple grayscale images from a binary string and store sequencely to a specific folder.

  Args:
    binary_string: The input binary string.
    width: The desired width of each image.
    height: The desired height of each image.
    max_pixels_per_image: The maximum number of pixels per image.
    output_folder: The name of the output folder.

  Returns:
    A list of filenames of the generated images.
  """

  os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist
  image_filenames = []
  image_count = 0

  while binary_string:
    image_name = os.path.join(output_folder, f"binary_image_{image_count:03d}.png")
    img = Image.new('L', (width, height), color=255)  # Create a new grayscale image
    pixels_to_process = min(max_pixels_per_image, len(binary_string))

    # Efficiently process pixels using a 1D array
    image_data = np.zeros((height, width), dtype=np.uint8)
    for i in range(pixels_to_process):
      if binary_string[i] == '0':
        image_data[i // width, i % width] = 0
      elif binary_string[i] == '1':
        image_data[i // width, i % width] = 255

    # Set pixels in the image using the efficient array
    img.putdata(image_data.flatten())

    img.save(image_name)
    image_filenames.append(image_name)
    image_count += 1

    binary_string = binary_string[pixels_to_process:]

  return image_filenames


# Example usage
# print("width = ",width)
# print("height = ",height)
# print("step = ",step)
# max_pixels_per_image=FS
# print("max_pixels_per_image = ",max_pixels_per_image)

image_filenames = create_grayscale_images_from_binary_string(base64_to_binary_text, 255, 255, 255*255)
print(f"\nGenerated image files length: {len(image_filenames)}")
print(f"Generated image files: { image_filenames}")


Generated image files length: 463
Generated image files: ['output_images/binary_image_000.png', 'output_images/binary_image_001.png', 'output_images/binary_image_002.png', 'output_images/binary_image_003.png', 'output_images/binary_image_004.png', 'output_images/binary_image_005.png', 'output_images/binary_image_006.png', 'output_images/binary_image_007.png', 'output_images/binary_image_008.png', 'output_images/binary_image_009.png', 'output_images/binary_image_010.png', 'output_images/binary_image_011.png', 'output_images/binary_image_012.png', 'output_images/binary_image_013.png', 'output_images/binary_image_014.png', 'output_images/binary_image_015.png', 'output_images/binary_image_016.png', 'output_images/binary_image_017.png', 'output_images/binary_image_018.png', 'output_images/binary_image_019.png', 'output_images/binary_image_020.png', 'output_images/binary_image_021.png', 'output_images/binary_image_022.png', 'output_images/binary_image_023.png', 'output_images/binary_image_0

In [ ]:
# @title OR 4.0 Creates N grayscale images from a binary string.

step = steps # pixel grid size
print("Step/Grid size is: ",step)

def create_grayscale_images_from_binary_string(binary_string, width, height, max_pixels_per_image, output_folder):
    """
    Creates multiple grayscale images from a binary string.

    Args:
        binary_string: The binary string data.
        width: Width of each image.
        height: Height of each image.
        max_pixels_per_image: Maximum number of pixels allowed per image.
        output_folder: The folder to save the images to.

    Returns:
        A list of filenames of the generated images.
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist
    image_filenames = []

    image_count = 0
    index = 0


    while index < len(binary_string):
        image_name = os.path.join(output_folder, f'binary_image_{image_count}.png')
        img = Image.new('L', (width, height), color=128)
        draw = ImageDraw.Draw(img)

        for y in range(0, height, step):
            for x in range(0, width, step):
                if index < len(binary_string):
                    color = 128

                    if binary_string[index] == '0':
                        color = 0
                    elif binary_string[index] == '1':
                        color = 255

                    draw.rectangle([x, y, x + step - 1, y + step - 1], fill=color)
                    index += 1
                else:
                    break  # Stop if we've reached the end of binary_string
            else:
                continue  # Continue if the inner loop wasn't broken
            break  # Break the outer loop if the inner loop finished naturally

        img.save(image_name)
        image_filenames.append(image_name)
        image_count += 1

    return image_filenames



Step/Grid size is:  3


In [ ]:
# Example usage (assuming you have a binary string)
# ... (your binary_string) ...
print("width = ",width)
print("height = ",height)
print("step = ",step)
max_pixels_per_image = FS
print("max_pixels_per_image = ",max_pixels_per_image)

output_folder="output_images_128"
print("output_folder = ",output_folder)

image_filenames = create_grayscale_images_from_binary_string(base64_to_binary_text, width, height, max_pixels_per_image, output_folder)
print(f"Generated image files: {image_filenames}")


width =  765
height =  765
step =  3
max_pixels_per_image =  585225
output_folder =  output_images_128
Generated image files: ['output_images_128/binary_image_0.png']


In [13]:
# @title 5.0 Generate a video (.mp4v) from a sequence of images.

import cv2

def create_video_from_images(image_folder, video_name, fps=24):
    """
    Generates a video file from a sequence of images.

    Args:
        image_folder (str): Path to the folder containing the images.
        video_name (str): Name of the output video file.
        fps (int, optional): Frames per second for the video. Defaults to 24.
    """

    images = [img for img in os.listdir(image_folder) if img.endswith((".jpg", ".jpeg", ".png"))]
    images.sort()  # Ensure images are processed in the correct order

    if not images:
        print(f"No images found in {image_folder}")
        return

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
    video = cv2.VideoWriter(video_name, fourcc, fps, (width, height))

    for image_path in images:
        img = cv2.imread(os.path.join(image_folder, image_path))
        video.write(img)

    video.release()
    print(f"Video '{video_name}' created successfully!")


In [15]:
# Replace with the actual path to your images in Google Drive
# image_folder = '/content/drive/MyDrive/Colab Notebooks/generated_images'
# image_folder = output_folder
image_folder = "output_images"

# Replace with the desired name for your output video file
# video_name = 'output_video.mp4
video_name = 'output_video_new.mp4'

create_video_from_images(image_folder, video_name, fps=24)

Video 'output_video_new.mp4' created successfully!




---



--- Uploa and store Data as Video file---


[Store on Youtube like services](https://www.youtube.com/@%E0%A6%86%E0%A6%AE%E0%A6%BF_%E0%A6%93_%E0%A6%86%E0%A6%AE%E0%A6%B0%E0%A6%BE)

--- Download and decode our data/files---

---



In [16]:
# @title 6.0 Extracts frames from a video file and saves them as PNG.

import cv2
import os

def extract_frames_from_video(video_path, output_dir):
  """
  Extracts frames from a video file and saves them as images.

  Args:
    video_path: Path to the video file.
    output_dir: Path to the directory where the frames will be saved.
  """

  try:
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist

    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0

    while success:
      frame_name = os.path.join(output_dir, f"frame_{count:04d}.png")  # Use 4-digit frame number for consistent naming
      cv2.imwrite(frame_name, image)
      success, image = vidcap.read()
      count += 1

    print(f"Successfully extracted {count} frames to {output_dir}")

  except Exception as e:
    print(f"Error extracting frames: {e}")



In [17]:
# Example usage:
# video_path = "/content/drive/MyDrive/Colab Notebooks/test_data/21.15.50.01.mp4"
video_path = "output_video_new.mp4"
output_dir = "output_images_128_re-generated"

extract_frames_from_video(video_path, output_dir)

Successfully extracted 463 frames to output_images_128_re-generated


In [20]:
sorted_image_list[:10]

['/content/output_images_128_re-generated/frame_0000.png',
 '/content/output_images_128_re-generated/frame_0001.png',
 '/content/output_images_128_re-generated/frame_0002.png',
 '/content/output_images_128_re-generated/frame_0003.png',
 '/content/output_images_128_re-generated/frame_0004.png',
 '/content/output_images_128_re-generated/frame_0005.png',
 '/content/output_images_128_re-generated/frame_0006.png',
 '/content/output_images_128_re-generated/frame_0007.png',
 '/content/output_images_128_re-generated/frame_0008.png',
 '/content/output_images_128_re-generated/frame_0009.png']

In [18]:
# @title  7.0 Reads all image file locations from a given directory and returns a sorted list of strings.

def get_sorted_image_paths(directory):
  """
  Reads all image file locations from a given directory and returns a sorted list of strings.

  Args:
    directory: The path to the directory containing the images.

  Returns:
    A sorted list of strings, where each string represents the full path to an image file.
  """
  image_extensions = [".jpg", ".jpeg", ".png"]  # Add more extensions if needed
  image_paths = []

  for filename in sorted(os.listdir(directory)):
      filepath = os.path.join(directory, filename)
      if os.path.isfile(filepath) and any(filepath.lower().endswith(ext) for ext in image_extensions):
          image_paths.append(filepath)

  return image_paths

# Example usage
# image_dir = "/path/to/your/image/directory"  # Replace with the actual directory path
sorted_image_list = get_sorted_image_paths("/content/output_images_128_re-generated")
print(len(sorted_image_list))
print(sorted_image_list)

463
['/content/output_images_128_re-generated/frame_0000.png', '/content/output_images_128_re-generated/frame_0001.png', '/content/output_images_128_re-generated/frame_0002.png', '/content/output_images_128_re-generated/frame_0003.png', '/content/output_images_128_re-generated/frame_0004.png', '/content/output_images_128_re-generated/frame_0005.png', '/content/output_images_128_re-generated/frame_0006.png', '/content/output_images_128_re-generated/frame_0007.png', '/content/output_images_128_re-generated/frame_0008.png', '/content/output_images_128_re-generated/frame_0009.png', '/content/output_images_128_re-generated/frame_0010.png', '/content/output_images_128_re-generated/frame_0011.png', '/content/output_images_128_re-generated/frame_0012.png', '/content/output_images_128_re-generated/frame_0013.png', '/content/output_images_128_re-generated/frame_0014.png', '/content/output_images_128_re-generated/frame_0015.png', '/content/output_images_128_re-generated/frame_0016.png', '/content

In [22]:
from PIL import Image
import os

# @title 8.0 Retrieves the original binary data from a set of grayscale images.

def get_binary_data_from_grayscale_images(image_filenames, width, height, step):
    """
    Retrieves the original binary data from a set of grayscale images.

    Args:
        image_filenames: A list of filenames of the grayscale images.
        width: Width of each image.
        height: Height of each image.
        step: The step size used when creating the images.

    Returns:
        The original binary string.
    """

    binary_string = ""

    for filename in image_filenames:
        img = Image.open(filename).convert('L')  # Open image in grayscale mode
        pixels = img.load()

        for y in range(0, height, step):
            for x in range(0, width, step):
                color = pixels[x, y]  # Get pixel value

                if color == 0:
                    binary_string += "0"
                elif color == 255:
                    binary_string += "1"
                else:
                    binary_string += ""  # Assume other values represent spaces/skip

    return binary_string



In [23]:
# Example usage:
# Assuming you have the list of image filenames from the previous step
# ... (image_filenames) ...

restored_binary_data = get_binary_data_from_grayscale_images(sorted_image_list, width, height, step=3)
print(restored_binary_data[:100])  # Print the restored binary data

if base64_to_binary_text[:200]==restored_binary_data[:200] and len(base64_to_binary_text)==len(restored_binary_data):
  print("success")
else:
  print("fail")

#

IndexError: image index out of range

**Failed means didnt got original data**

Problem:

Solution:

In [ ]:
print(  len(base64_to_binary_text))
print(len(restored_binary_data))
print(restored_binary_data.count("0"))
print(restored_binary_data.count("1"))

2944
1222
1031
191




---



---



In [ ]:
from PIL import Image

def open_image(image_path):
  """
  Opens an image from the specified location.

  Args:
    image_path: The path to the image file.

  Returns:
    A PIL Image object representing the opened image.

  Raises:
    FileNotFoundError: If the image file is not found.
    OSError: If an error occurs while opening the image.
  """

  try:
    img = Image.open(image_path)
    return img
  except FileNotFoundError:
    raise FileNotFoundError(f"Image file not found: {image_path}")
  except OSError as e:
    raise OSError(f"Error opening image: {image_path} ({e})")

# Example usage
image_path =  "output_images_128_re-generated/frame_4040.png"
print(image_path)

img = open_image("output_images_128_re-generated/frame_4040.png")
img.show()  # Display the image


output_images_128_re-generated/frame_4040.png




---



---



#Decode (Ext)

In [ ]:
# @title Converts a binary string to a base64 encoded string.
def binary_to_base64(binary_string):
    """Converts a binary string to a base64 encoded string."""
    try:
        # Ensure the binary string has a length divisible by 8
        if len(binary_string) % 8 != 0:
            print("Warning: Binary string length is not a multiple of 8. Padding with zeros.")
            binary_string += '0' * (8 - (len(binary_string) % 8))  # Pad with zeros

        decoded_bytes = bytearray()
        for i in range(0, len(binary_string), 8):
            byte_string = binary_string[i:i+8]
            decoded_bytes.append(int(byte_string, 2))

        base64_string = base64.b64encode(decoded_bytes).decode('utf-8')
        return base64_string
    except Exception as e:
        print(f"An error occurred: {e}")
        return None



827300
827300
success


'UEsDBBQAAAAIAHs3MFoE6PblHawAALuAAQAOAAAAYm9va21hcmtzLmh0bWzsu1mzrEh6rnmvX7GVF33Mmk4BAQGEVFltzEMwz3AjY57nmV/f7EpVSepz7JyWzFrdF7lsbyPC3XF33L/heZ21/vT3jEbbgc7+UFnbokmd/ZUahqaL5ubXvGqzX+E//92f/v7XX3/YZbX8'

In [ ]:
restored_binary_to_base64_text = binary_to_base64(restored_binary_data)

print(len(restored_binary_to_base64_text))
print(len(zip_to_base64_text))
if zip_to_base64_text[:200]== restored_binary_to_base64_text[:200]:
  print("success")
else:
  print("fail")

restored_binary_to_base64_text[:200]

In [ ]:

# @title Converts a base64 back to a zip file.
def base64_to_zip(text_data, zip_filename):
    """Converts a base64 encoded string back to a zip file."""
    try:
        decoded_zip = base64.b64decode(text_data)
        with open(zip_filename, "wb") as zip_file:
            zip_file.write(decoded_zip)
        print(f"Successfully created {zip_filename}")
    except Exception as e:
        print(f"An error occurred: {e}")


restored_archive="restored_archive.zip"
base64_to_zip(restored_binary_to_base64_text,restored_archive )

Successfully created restored_archive.zip


In [ ]:
# @title Unzips a zip archive to a specified directory.


def unzip_files(zip_filename, output_dir):
  """
  Unzips a zip archive to a specified directory.

  Args:
    zip_filename: The path to the zip file to unzip.
    output_dir: The directory to extract the contents to.
  """
  try:
    # Avoid changing the working directory (os.chdir) unnecessarily
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
      os.makedirs(output_dir, exist_ok=True)
      print(f"Unzipping {zip_filename} to {output_dir}")
      zipf.extractall(output_dir)
    print(f"Successfully extracted {zip_filename} to {output_dir}")

  except FileNotFoundError:
    print(f"Error: Zip file not found: {zip_filename}")
  except zipfile.BadZipFile:
    print(f"Error: Invalid zip file: {zip_filename}")
  except Exception as e:
    print(f"An error occurred: {e}")



# Example Usage:
print("Files in the current directory:"+ restored_archive)
# zip_filename = restored_archive  # Replace with the actual zip file path
output_dir = "unzipprd_out_dir"  # Replace with your desired extraction directory
unzip_files(restored_archive, output_dir)

Files in the current directory:restored_archive.zip
Unzipping restored_archive.zip to unzipprd_out_dir
Successfully extracted restored_archive.zip to unzipprd_out_dir




---





---

execute

---



In [ ]:
# @title Example usage (assuming you have a base64 encoded zip string)
print("\nbase64 representation :")
zip_to_base64_string = zip_to_base64("my_archive.zip") #Replace with your zip file
print(zip_to_base64_string[:100])


# Example usage (assuming you have a binary string)
# ... (previous code) ...


if zip_to_base64_string:
    binary_data = base64_to_binary(zip_to_base64_string)
    if binary_data:
        print("Binary representation:")
        print(binary_data[:100])  # Print the first 100 characters of the binary data
        print(binary_data.count)

        # Convert binary back to base64
        restored_base64 = binary_to_base64(binary_data)
        if restored_base64:
            print("\nRestored base64 string:")
            print(restored_base64[:100]) # Print the first 100 characters for verification
            print(restored_base64.count)
            #Convert to zip file
            base64_to_zip(restored_base64, "restored_archive.zip")
        else:
            print("Failed to convert binary back to base64.")
    else:
        print("Failed to convert base64 to binary.")
else:
    print("Failed to convert zip to base64.")


base64 representation string:
UEsDBBQAAAAIAPKlK1pwKXSnRb0HAB1ICAAXAAAASU1HXzIwMTUxMTIxXzEwMjAwNC5qcGfsvXk8VV//8L3POaYoSiIRDaSSCkkU
Binary representation:
0101000001001011000000110000010000010100000000000000000000000000000010000000000011110010101001010010
<built-in method count of str object at 0x55e577431bf0>

Restored base64 string:
UEsDBBQAAAAIAPKlK1pwKXSnRb0HAB1ICAAXAAAASU1HXzIwMTUxMTIxXzEwMjAwNC5qcGfsvXk8VV//8L3POaYoSiIRDaSSCkkU
<built-in method count of str object at 0x55e576c8d7b0>
Successfully created restored_archive.zip


In [ ]:
# @title Converts a base64 encoded zip string to binary data.
# now create a function to convert encoded_zip into binary also create a function to convert back into previous data type


# @title Converts a base64 encoded zip string to binary data.
def encoded_zip_to_binary(encoded_zip):
  """Converts a base64 encoded zip string to binary data.

  Args:
    encoded_zip: The base64 encoded zip string.

  Returns:
    The binary data representing the zip file, or None if an error occurs.
  """
  try:
    zip_bytes = base64.b64decode(encoded_zip)
    return zip_bytes
  except Exception as e:
    print(f"Error decoding base64 string: {e}")
    return None

# @title Converts a base64 encoded zip string to binary data.
def binary_to_encoded_zip(zip_bytes):
  """Converts binary zip data to a base64 encoded string.

  Args:
    zip_bytes: The binary data representing the zip file.

  Returns:
    The base64 encoded string, or None if an error occurs.
  """
  try:
    encoded_zip = base64.b64encode(zip_bytes).decode('utf-8')
    return encoded_zip
  except Exception as e:
    print(f"Error encoding to base64 string: {e}")
    return None

bData=zip_to_base64_text("/content/zipped_out_dir/my_archive.zip")
bData[:1100]

bData=encoded_zip_to_binary(bData)

bData[:100]



---

